## Ноутбук запускается в Yandex DataSphere

In [1]:
#%pip install --upgrade pip
#%pip install yandexcloud --upgrade
#%pip install yandex-cloud-ml-sdk --upgrade
#%pip install scikit-learn --upgrade

In [2]:
import os
import pandas as pd
import numpy as np
from yandex_cloud_ml_sdk import YCloudML

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.metrics import cohen_kappa_score

In [3]:
df = pd.read_excel('marked_data.xlsx')
df['labels'] = df['labels'] + 1

# Zero-shot

In [36]:
df_test = df[9900:]

In [37]:
sdk = YCloudML(
        folder_id=os.environ['FOLDER_ID'],
        auth=os.environ['API_KEY'],
    )
    
model = sdk.models.text_classifiers("yandexgpt").configure(
        task_description="определи тип интента",
        labels=["негативный", "нейтральный", "позитивный"],
    )

In [38]:
pred = []
p_neg = []
p_neu = []
p_pos = []
for i in df_test.index:
    request_text = df_test['text'][i]
    result = model.run(request_text)
    pred.append(np.argmax([result[0]['confidence'], result[1]['confidence'], result[2]['confidence']]))
    p_neg.append(result[0]['confidence'])
    p_neu.append(result[1]['confidence'])
    p_pos.append(result[2]['confidence'])

In [39]:
df_test_res0 = df_test.copy()
df_test_res0['predict'] = pred
df_test_res0['Pneg'] = p_neg
df_test_res0['Pneutral'] = p_neu
df_test_res0['Ppos'] = p_pos
df_test_res0['Ppos-Pneg'] = df_test_res0['Ppos'] - df_test_res0['Pneg']
df_test_res0

,post_id,text,labels,predict,Pneg,Pneutral,Ppos,Ppos-Pneg
9900,5910239,Сохранять деньги в долларах??? Можно сильно п...,0,0,9.999989e-01,6.312644e-08,4.163640e-09,-0.999999
9901,4774600,"молодцы ер , лучше на помойку...",0,0,9.999990e-01,3.242647e-09,4.074533e-10,-0.999999
9902,333368,Херню бредовую он высказал.,0,0,9.999990e-01,3.940879e-11,2.871968e-11,-0.999999
9903,1552124,"Доллар, валюта недружественной страны, так поч...",0,0,9.999990e-01,4.501969e-09,1.804805e-09,-0.999999
9904,1602851,🤣,2,2,2.265267e-11,1.820465e-10,9.999990e-01,0.999999
...,...,...,...,...,...,...,...,...
9995,1139301,"Не, ну это вообще пи.дец...!😡😤😡😤",2,0,9.999990e-01,8.583595e-13,3.633512e-12,-0.999999
9996,5343030,"Чего он сказал-то? Как назвал , оскорбил? Ниче...",0,0,9.999990e-01,4.572865e-09,7.067716e-10,-0.999999
9997,5595571,"Они все , включая и эту НаЕбулину , живут в Пу...",0,0,9.999990e-01,5.211407e-10,1.092370e-10,-0.999999
9998,6590679,"ну так власятм спасибо скажите , это их рук дело",0,0,9.999989e-01,8.362895e-08,2.491456e-08,-0.999999


In [40]:
print('accuracy', accuracy_score(df_test_res0['labels'], df_test_res0['predict']))
print('macro_precision', precision_score(df_test_res0['labels'], df_test_res0['predict'], average='macro'))
print('macro_recall', recall_score(df_test_res0['labels'], df_test_res0['predict'], average='macro'))
print('macro_f1', f1_score(df_test_res0['labels'], df_test_res0['predict'], average='macro'))
print('RMSE for Ppos-Pneg', root_mean_squared_error(df_test_res0['labels']-1, df_test_res0['Ppos-Pneg']))
print('MAE for Ppos-Pneg', mean_absolute_error(df_test_res0['labels']-1, df_test_res0['Ppos-Pneg']))

accuracy 0.6
macro_precision 0.5666666666666667
macro_recall 0.5287581699346405
macro_f1 0.529004329004329
RMSE for Ppos-Pneg 0.871624515169865
MAE for Ppos-Pneg 0.5220409507429449


In [41]:
print('precision by class', precision_score(df_test_res0['labels'], df_test_res0['predict'], average=None))
print('recall by class', recall_score(df_test_res0['labels'], df_test_res0['predict'], average=None))
print('f1 by class', f1_score(df_test_res0['labels'], df_test_res0['predict'], average=None))

precision by class [0.625 0.7   0.375]
recall by class [0.83333333 0.4        0.35294118]
f1 by class [0.71428571 0.50909091 0.36363636]


In [42]:
confusion_matrix(df_test_res0['labels'], df_test_res0['predict'], labels=[0,1,2])

array([[40,  5,  3],
       [14, 14,  7],
       [10,  1,  6]])

In [43]:
cohen_kappa_score(df_test_res0['labels'], df_test_res0['predict'], labels=None, weights= 'quadratic', sample_weight=None)

0.3144741409232906

In [44]:
df_test_res0.to_excel('YGPT_Zero-shot_9900-9999.xlsx')